In [38]:
import open3d as o3d
import numpy as np

In [34]:
def load_off_file(file_path):
    """
    Load a .off file and return it as a point cloud.
    """
    mesh = o3d.io.read_triangle_mesh(file_path)  # Read the .off file as a mesh
    print(f"Loaded {file_path}, vertices: {len(mesh.vertices)}, triangles: {len(mesh.triangles)}")
    
    if not mesh.has_triangles():
        print(f"Error: {file_path} does not contain valid triangles.")
        return None
    
    pcd = mesh.sample_points_poisson_disk(number_of_points=10000)  # Sample points from the mesh
    return pcd

In [35]:
# Test ICP registration on two point clouds
source_pcd = load_off_file("Dataset/ModelNet10/bathtub/train/bathtub_0001.off")
target_pcd = load_off_file("Dataset/ModelNet10/bathtub/train/bathtub_0002.off")

Loaded Dataset/ModelNet10/bathtub/train/bathtub_0001.off, vertices: 3514, triangles: 3546
Loaded Dataset/ModelNet10/bathtub/train/bathtub_0002.off, vertices: 700, triangles: 664


In [41]:
def apply_icp(source, target, threshold=0.05):  # Increased threshold for flexibility
    """
    Apply ICP (Iterative Closest Point) for point cloud registration.
    """
    print(f"Source Point Cloud type: {type(source)}")
    print(f"Target Point Cloud type: {type(target)}")
    
    # Check the number of points in each point cloud
    print(f"Source Point Cloud has {len(source.points)} points.")
    print(f"Target Point Cloud has {len(target.points)} points.")

    # Identity matrix as the initial transformation
    init_transformation = np.eye(4)
    print(f"Initial Transformation Matrix:\n{init_transformation}")

    # Define the ICP method (Point-to-Point)
    estimation_method = o3d.pipelines.registration.TransformationEstimationPointToPoint(with_scaling=False)
    
    # Perform ICP registration with the defined method
    icp_result = o3d.pipelines.registration.registration_icp(
        source, target, threshold,
        init_transformation,  # Identity matrix as initial transformation
        estimation_method,    # Explicitly specify the estimation method
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=200)
    )
    
    # Return the ICP result
    return icp_result


In [42]:
if source_pcd and target_pcd:
    # Apply ICP
    icp_result = apply_icp(source_pcd, target_pcd)

    # Visualize the ICP result
    source_temp = source_pcd.transform(icp_result.transformation)
    o3d.visualization.draw_geometries([source_temp, target_pcd], window_name="ICP Registered Point Clouds")

    # Print the ICP results (transformation matrix)
    print("ICP Transformation Matrix:")
    print(icp_result.transformation)
else:
    print("One or both point clouds are invalid, skipping ICP.")


Source Point Cloud type: <class 'open3d.cpu.pybind.geometry.PointCloud'>
Target Point Cloud type: <class 'open3d.cpu.pybind.geometry.PointCloud'>
Source Point Cloud has 10000 points.
Target Point Cloud has 10000 points.
Initial Transformation Matrix:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
ICP Transformation Matrix:
[[ 0.243409   -0.94377874  0.22368226 13.63727536]
 [-0.94400372 -0.28347689 -0.16881297 21.19975016]
 [ 0.22273085 -0.17006629 -0.95993147 13.97086528]
 [ 0.          0.          0.          1.        ]]


In [43]:
def save_point_cloud(pcd, file_path):
    """
    Save the point cloud to a .pcd file.
    """
    o3d.io.write_point_cloud(file_path, pcd)
    print(f"Saved the point cloud to: {file_path}")

# Test saving the registered point cloud
output_path = "Registered_Point_Clouds/test_bathtub_0001_registered.pcd"
if source_pcd:
    save_point_cloud(source_temp, output_path)


Saved the point cloud to: Registered_Point_Clouds/test_bathtub_0001_registered.pcd


In [44]:
def visualize_registered_point_clouds(output_path):
    """
    Visualize the saved point cloud.
    """
    saved_pcd = o3d.io.read_point_cloud(output_path)
    o3d.visualization.draw_geometries([saved_pcd], window_name="Saved Point Cloud")

# Test visualizing the saved point cloud
visualize_registered_point_clouds(output_path)

In [45]:
import os
import open3d as o3d
import numpy as np

def apply_icp(source, target, threshold=0.05):
    """
    Apply ICP (Iterative Closest Point) for point cloud registration.
    """
    print(f"Source Point Cloud type: {type(source)}")
    print(f"Target Point Cloud type: {type(target)}")
    
    # Check the number of points in each point cloud
    print(f"Source Point Cloud has {len(source.points)} points.")
    print(f"Target Point Cloud has {len(target.points)} points.")

    # Identity matrix as the initial transformation
    init_transformation = np.eye(4)
    print(f"Initial Transformation Matrix:\n{init_transformation}")

    # Define the ICP method (Point-to-Point)
    estimation_method = o3d.pipelines.registration.TransformationEstimationPointToPoint(with_scaling=False)
    
    # Perform ICP registration with the defined method
    icp_result = o3d.pipelines.registration.registration_icp(
        source, target, threshold,
        init_transformation,  # Identity matrix as initial transformation
        estimation_method,    # Explicitly specify the estimation method
        o3d.pipelines.registration.ICPConvergenceCriteria(max_iteration=200)
    )
    
    # Return the ICP result
    return icp_result

def evaluate_icp_registration(icp_result, source, target):
    """
    Evaluate the quality of the ICP registration by calculating the fitness score and inlier RMSE.
    """
    fitness = icp_result.fitness  # Fitness score
    inlier_rmse = icp_result.inlier_rmse  # Inlier RMSE
    
    # Optionally, print the values
    print(f"Fitness: {fitness:.4f}")
    print(f"Inlier RMSE: {inlier_rmse:.4f}")
    
    return fitness, inlier_rmse

def visualize_registration(source, target, transformation):
    """
    Visualize the source and target point clouds after applying the ICP transformation to the source point cloud.
    """
    # Apply the transformation to the source point cloud
    source_temp = source.transform(transformation)
    
    # Visualize the point clouds
    o3d.visualization.draw_geometries([source_temp, target], window_name="ICP Registered Point Clouds")

def save_point_cloud(pcd, file_path):
    """
    Save the point cloud to a .pcd file.
    """
    o3d.io.write_point_cloud(file_path, pcd)
    print(f"Saved the point cloud to: {file_path}")

def visualize_registered_point_clouds(output_path):
    """
    Visualize the saved point cloud.
    """
    saved_pcd = o3d.io.read_point_cloud(output_path)
    o3d.visualization.draw_geometries([saved_pcd], window_name="Saved Point Cloud")

def register_all_point_clouds(data_dir, output_dir, threshold=0.05):
    """
    Registers all point clouds in the given directory using ICP.
    """
    categories = os.listdir(data_dir)  # List of object categories (e.g., bathtub, bed, chair, etc.)
    
    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    for category in categories:
        category_folder = os.path.join(data_dir, category, 'train')  # Assuming point clouds are in 'train' folder
        if os.path.isdir(category_folder):
            print(f"Processing category: {category}")
            
            # Get all .off files in the category folder
            off_files = [f for f in os.listdir(category_folder) if f.endswith('.off')]
            off_files.sort()  # Sort the files to maintain order
            
            # Register each consecutive pair of .off files in the category
            for i in range(len(off_files) - 1):
                # Load source and target point clouds from .off files
                source_off_path = os.path.join(category_folder, off_files[i])
                target_off_path = os.path.join(category_folder, off_files[i + 1])
                
                source_pcd = load_off_file(source_off_path)  # Convert .off to point cloud
                target_pcd = load_off_file(target_off_path)
                
                # Apply ICP only if both point clouds are valid
                if source_pcd and target_pcd:
                    icp_result = apply_icp(source_pcd, target_pcd, threshold)
                    
                    # Print ICP results
                    print(f"ICP Registration between {off_files[i]} and {off_files[i + 1]}:")
                    fitness, inlier_rmse = evaluate_icp_registration(icp_result, source_pcd, target_pcd)
                    print(f"Fitness Score: {fitness:.4f}, Inlier RMSE: {inlier_rmse:.4f}")

                    # Visualize the result
                    visualize_registration(source_pcd, target_pcd, icp_result.transformation)

                    # Save the registered source point cloud
                    output_path = os.path.join(output_dir, f"{category}_{off_files[i].replace('.off', '_registered.pcd')}")
                    save_point_cloud(source_pcd, output_path)
                    print(f"Saved registered point cloud: {output_path}")

                    # Visualize the saved point cloud
                    visualize_registered_point_clouds(output_path)

            print(f"Finished processing category: {category}")


In [46]:
data_dir = "Dataset/ModelNet10"
output_dir = "Registered_Point_Clouds"
register_all_point_clouds(data_dir, output_dir, threshold=0.05)

Processing category: bathtub
Loaded Dataset/ModelNet10\bathtub\train\bathtub_0001.off, vertices: 3514, triangles: 3546
Loaded Dataset/ModelNet10\bathtub\train\bathtub_0002.off, vertices: 700, triangles: 664
Source Point Cloud type: <class 'open3d.cpu.pybind.geometry.PointCloud'>
Target Point Cloud type: <class 'open3d.cpu.pybind.geometry.PointCloud'>
Source Point Cloud has 10000 points.
Target Point Cloud has 10000 points.
Initial Transformation Matrix:
[[1. 0. 0. 0.]
 [0. 1. 0. 0.]
 [0. 0. 1. 0.]
 [0. 0. 0. 1.]]
ICP Registration between bathtub_0001.off and bathtub_0002.off:
Fitness: 0.0000
Inlier RMSE: 0.0000
Fitness Score: 0.0000, Inlier RMSE: 0.0000
Saved the point cloud to: Registered_Point_Clouds\bathtub_bathtub_0001_registered.pcd
Saved registered point cloud: Registered_Point_Clouds\bathtub_bathtub_0001_registered.pcd
Loaded Dataset/ModelNet10\bathtub\train\bathtub_0002.off, vertices: 700, triangles: 664
Loaded Dataset/ModelNet10\bathtub\train\bathtub_0003.off, vertices: 14963,

KeyboardInterrupt: 